This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/contact.html).  I recommend having both windows open, side-by-side!


In [ ]:
import matplotlib.pyplot as plt
import mpld3
from IPython.display import display
import numpy as np
from pydrake.all import (
    DiagramBuilder,
    HybridMultibodyCollocation,
    MeshcatVisualizer,
    MultibodyPlant,
    MultibodyPositionToGeometryPose,
    Parser,
    RobotDiagramBuilder,
    Role,
    SceneGraph,
    Simulator,
    Solve,
    StartMeshcat,
    TrajectorySource,
    eq,
)

from underactuated import running_as_notebook

if running_as_notebook:
    mpld3.enable_notebook()

np.set_printoptions(suppress=True, precision=6)

In [ ]:
meshcat = StartMeshcat()

# Hybrid Multibody Collocation



In [ ]:
def ball_on_the_floor():
    ball_floor_xml = """
<mujoco model="test">
  <worldbody>
    <geom name="floor" type="box" size="5 5 5" pos="0 0 -5" />
    <body name="ball">
      <joint type="slide" name="z" axis="0 0 1" />
      <geom name="ball" type="sphere" size="1" mass="1" />
    </body>
  </worldbody>
</mujoco>
"""

    builder = RobotDiagramBuilder()
    builder.parser().AddModelsFromString(ball_floor_xml, "xml")
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = diagram.plant().GetMyContextFromRoot(context)

    kTimeStep = 0.1
    hybrid = HybridMultibodyCollocation(diagram, context, kTimeStep, kTimeStep)
    prog = hybrid.prog()

    kNumTimeSteps = 2
    mode = hybrid.AddMode(
        "ground", kNumTimeSteps, hybrid.GetContactPairCandidates()
    )

    # zdot = 0.
    prog.AddBoundingBoxConstraint(0, 0, mode.initial_state()[1])
    prog.AddBoundingBoxConstraint(0, 0, mode.final_state()[1])

    # Constant force (otherwise there is a manifold of solutions, since only the
    # average force over an interval must balance gravity).
    prog.AddLinearConstraint(
        eq(mode.AllContactForces(0), mode.AllContactForces(1))
    )

    result = Solve(prog)
    print(result.is_success())

    traj = hybrid.ReconstructStateTrajectory(result)
    t = np.linspace(traj.start_time(), traj.end_time(), 1000)
    plt.plot(
        t,
        traj.vector_values(t).T,
        label=["z", "zdot"],
    )

    mg = (
        diagram.plant().CalcTotalMass(plant_context)
        * diagram.plant().gravity_field().gravity_vector()
    )
    prog.SetInitialGuess(prog.decision_variables(), np.zeros(prog.num_vars()))
    prog.SetInitialGuess(mode.time_step(0), [kTimeStep])
    prog.SetInitialGuess(mode.state(0), [1, 0])
    prog.SetInitialGuess(mode.state(1), [1, 0])
    prog.SetInitialGuess(mode.AllContactForces(0), mg)
    prog.SetInitialGuess(mode.AllContactForces(1), mg)
    print(prog.EvalBindingAtInitialGuess(prog.generic_constraints()[0]))


ball_on_the_floor()

In [ ]:
def Animate(xml, hybrid, result):
    x_traj = hybrid.ReconstructStateTrajectory(result)

    builder = DiagramBuilder()
    plant = MultibodyPlant(0.0)
    scene_graph = builder.AddSystem(SceneGraph())
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    Parser(plant, scene_graph).AddModelsFromString(xml, "xml")
    plant.Finalize()

    source = builder.AddSystem(TrajectorySource(x_traj))
    pos_to_pose = builder.AddSystem(
        MultibodyPositionToGeometryPose(plant, input_multibody_state=True)
    )
    builder.Connect(source.get_output_port(0), pos_to_pose.get_input_port())
    builder.Connect(
        pos_to_pose.get_output_port(),
        scene_graph.get_source_pose_port(plant.get_source_id()),
    )

    meshcat.Set2dRenderMode(xmin=-5, xmax=5, ymin=-2, ymax=5)
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    diagram = builder.Build()
    simulator = Simulator(diagram)
    visualizer.StartRecording(False)
    simulator.AdvanceTo(x_traj.end_time())
    visualizer.PublishRecording()


def BallDrop():
    xml = """
<mujoco model="test">
<worldbody>
    <geom name="ramp1" type="box" size="50 50 50" pos="0 0 -57.735"
        euler="0 30 0" friction="5 0 0" />
    <geom name="ramp2" type="box" size="50 50 50" pos="0 0 -50" 
        friction="5 0 0" />
    <body name="body">
    <joint type="slide" name="x" axis="1 0 0" />
    <joint type="slide" name="z" axis="0 0 1" />
    <joint type="hinge" name="θ" axis="0 1 0" />
    <geom name="ball" type="sphere" size="1" mass="1"
        rgba="0.6 0.2 0.2 1" friction="5 0 0" />
    </body>
</worldbody>
</mujoco>"""

    builder = RobotDiagramBuilder()
    builder.parser().AddModelsFromString(xml, "xml")
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant = diagram.plant()
    plant_context = plant.GetMyContextFromRoot(context)

    kMinTimeStep = 0.05
    kMaxTimeStep = 1.0
    hybrid = HybridMultibodyCollocation(
        diagram, context, kMinTimeStep, kMaxTimeStep
    )
    prog = hybrid.prog()
    inspector = hybrid.model_inspector()
    ball = inspector.GetGeometryIdByName(
        plant.GetBodyFrameIdOrThrow(plant.GetBodyByName("body").index()),
        Role.kProximity,
        "test::ball",
    )
    ramp1 = inspector.GetGeometryIdByName(
        inspector.world_frame_id(), Role.kProximity, "ramp1"
    )
    ramp2 = inspector.GetGeometryIdByName(
        inspector.world_frame_id(), Role.kProximity, "ramp2"
    )

    kNumTimeSteps = 5
    aerial_phase = hybrid.AddMode("aerial", kNumTimeSteps, set())
    aerial_phase.AddEqualTimeIntervalsConstraints()
    ramp1_phase = hybrid.AddModeWithInelasticImpact(
        "ramp1", 3, {(ball, ramp1)}
    )
    ramp1_phase.AddEqualTimeIntervalsConstraints()
    # ramp2_phase = hybrid.AddModeWithInelasticImpact(
    #    "ramp2", kNumTimeSteps, {(ball, ramp2)}
    # )
    # ramp2_phase.AddEqualTimeIntervalsConstraints()

    x0_min = [-2, 5, 0, 0, 0, 0]
    x0_max = [-2, 5, 0, 0, 0, 0]
    prog.AddBoundingBoxConstraint(
        x0_min, x0_max, hybrid.mode(0).initial_state()
    )

    #    for c in prog.generic_constraints():
    #        print(c)
    result = Solve(prog)
    #    for v in prog.decision_variables():
    #        print(f"{v.get_name()} = {result.GetSolution(v)}")

    if not result.is_success():
        print(result.GetInfeasibleConstraintNames(prog))
        print(result.EvalBinding(result.GetInfeasibleConstraints(prog)[0]))
        print(
            result.GetInfeasibleConstraints(prog)[0].evaluator().upper_bound()
        )

    Animate(xml, hybrid, result)

    plant_context.SetContinuousState(
        result.GetSolution(ramp1_phase.initial_state())
    )
    M = plant.CalcMassMatrixViaInverseDynamics(plant_context)
    vplus = result.GetSolution(ramp1_phase.initial_state()[3:])
    vminus = result.GetSolution(aerial_phase.final_state()[3:])
    # print(vplus)
    # print(vminus)
    # print(M @ (vplus - vminus))

    ramp1_traj = ramp1_phase.ReconstructStateTrajectory(result)
    # print(ramp1_traj.vector_values(ramp1_traj.get_segment_times()))
    ramp1_force = ramp1_phase.ReconstructContactForceTrajectory(
        result, (ball, ramp1)
    )
    # print(ramp1_force.vector_values(ramp1_force.get_segment_times()))
    #    ramp2_force = ramp2_phase.ReconstructContactForceTrajectory(
    #        result, (ball, ramp2)
    #    )
    #    print(ramp2_force.vector_values(ramp2_force.get_segment_times()))

    #    print(result.GetSolution(ramp1_phase.final_state().T))
    #    print(result.GetSolution(ramp2_phase.initial_state().T))

    traj = hybrid.ReconstructStateTrajectory(result)
    t = np.linspace(traj.start_time(), traj.end_time(), 1000)
    plt.plot(
        t,
        traj.vector_values(t)[:6, :].T,
        label=["x", "z", "θ", "xdot", "zdot", "θdot"],
    )
    f_ball_ramp1_traj = hybrid.ReconstructContactForceTrajectory(
        result, (ball, ramp1)
    )
    # plt.plot(
    #    t,
    #    f_ball_ramp1_traj.vector_values(t)[[0, 2], :].T,
    #    label=["f_x", "f_z"],
    # )
    plt.legend()
    display(mpld3.display())


BallDrop()

In [ ]:
M = np.mat("[1, 0, 0; 0, 1, 0; 0, 0, 0.4]")
J = np.mat(
    "[-1,        0 ,       0;  0,        0 ,      -1; 0.866025,        0 ,    -0.5]"
).T
vminus = np.array([0, -7.26561, 0]).reshape((3, 1))
Lambda = np.array([-1.8754, 0, -6.18284]).reshape((3, 1))
print(vminus + np.linalg.inv(M) @ J.T @ Lambda)
vplus = np.array([1.8754, -1.08276, 2.16553]).reshape((3, 1))
nhat = np.array([-0.5, -0, -0.866025]).reshape((3, 1))
print(J @ vplus)

In [ ]:
Lambda2 = (
    -np.linalg.pinv(J @ np.linalg.inv(M) @ J.T) @ J @ vminus.reshape((3, 1))
)
print(Lambda2)
vplus2 = vminus + np.linalg.inv(M) @ J.T @ Lambda2
print(M @ (vplus2 - vminus) - J.T @ Lambda2)
print(J @ vplus2)
print(Lambda2.T @ nhat)
# fᵀ((1+μ²)nnᵀ - I)f ≥ 0.
print(Lambda2.T @ ((1 + 25) * nhat @ nhat.T - np.eye(3)) @ Lambda2)

In [ ]:
n = np.array([-0.5, -0, -0.866025])
f0 = np.array([0.33246499627872467, 0.0, -2.2270108775641946])
fbar = np.array([-4.836512236164078, 0, -11.179942067321896])